In [1]:
import sys
import os

from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from TSNE_Algorithm.tsneAlgo import TsnePlot
from plotly.graph_objects import Figure, Scatter
import plotly

In [2]:
styles = """
        QWidget {
          Background: '#a8d0e6';
        }
        QLabel {
          padding-top: 10px;
          padding-bottom: 30px;
          font-family: 'Helvetica';
          font-size: 15px  
        }
        QPushButton {
          Background: #80aaff;
          color: 'black';
          font-size: 15px;
          border-radius: 20px;
          border: 2px solid '#6699ff'
        }
        QPushButton:hover {
          Background: #b3ccff;
          color: 'black';
          font-size: 15px;
          border-radius: 20px;
        }
        """

recentFileList = []
def readRecentFiles():
    global recentFileList
    file1 = open('recentFiles.txt', 'r')
    Lines = file1.readlines()
    non_empty_lines = [line for line in Lines if line.strip() != ""]
    for line in non_empty_lines:
        recentFileList.append(line)
        
def writeRecentFiles():
    global recentFileList
    file1 = open('recentFiles.txt', 'w')
    i = 0
    while recentFileList and i < 10:
        file1.write(recentFileList.pop())
        i += 1
    file1.close()


In [3]:
class UI(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        global recentFileList
        readRecentFiles()

        self.resize(1500, 750)
        self.center()
        # self.setWindowTitle('no title')
        self.setStyleSheet(styles)
        
        label1 = QLabel('<h1>t-SNE Clustering Analysis</h1>')
        label1.setAlignment(Qt.AlignHCenter)
        
        label2 = QLabel('<h3>Upload a file to run the t-SNE algorithm on:</h3>')
        label2.setAlignment(Qt.AlignCenter)

        
        openFileBtn = QPushButton("Open File", self)
        openFileBtn.setToolTip('Click here to upload a file')
        openFileBtn.setFixedSize(220, 50)

        helpBtn = QPushButton("Help", self)
        helpBtn.setToolTip('Click here to read instructions')
        helpBtn.setFixedSize(220, 50)
        
        quitBtn = QPushButton("Quit", self)
        quitBtn.setToolTip('Click here to quit')
        quitBtn.setFixedSize(220, 50)
        
        recentFileListWidget = QListWidget()
        recentFileListWidget.setFixedSize(450, 450)
        recentFileListWidget.setStyleSheet('QListWidget'
                                            '{'
                                            'border: 3px solid #d7eaf4;'
                                            'padding: 10px;'
                                            'border-radius: 15px'
                                            '}')
        
        for line in recentFileList:
            QListWidgetItem(os.path.basename(line), recentFileListWidget)
        
        # tempWidget = QListWidget()
        # tempWidget.setFixedSize(450, 450)

        placeholder = QPushButton(' ', self)
        placeholder.setFixedSize(450, 450)
        placeholder.setStyleSheet('background: #a8d0e6;'
                                  'border-color: #a8d0e6;')
        
        outer_layout = QHBoxLayout()
        layout = QVBoxLayout()
        layout.addWidget(label1)
        layout.addStretch()
        layout.addWidget(label2)
        layout.addWidget(openFileBtn, alignment=Qt.AlignCenter)
        layout.addWidget(helpBtn, alignment=Qt.AlignCenter)
        layout.addWidget(quitBtn, alignment=Qt.AlignCenter)
        layout.addStretch()

        outer_layout.addWidget(placeholder, alignment=Qt.AlignLeft)
        outer_layout.addLayout(layout)
        outer_layout.addWidget(recentFileListWidget, alignment=Qt.AlignRight)

        self.setLayout(outer_layout)
        openFileBtn.clicked.connect(self.open)
        helpBtn.clicked.connect(self.open_instructions_window)
        quitBtn.clicked.connect(self.close)
        
        self.show()

    def center(self):
        # geometry of the main window
        qr = self.frameGeometry()

        # center point of screen
        cp = QDesktopWidget().availableGeometry().center()

        # move rectangle's center point to screen's center point
        qr.moveCenter(cp)

        # top left of rectangle becomes top left of window centering it
        self.move(qr.topLeft())
        
    def open(self):
        global recentFileList
        path = QFileDialog.getOpenFileName(self, 'Open a file', '',
                                        'All Files (*.*)')
        if path != ('', ''):
            print("File path : "+ path[0])
            
            pathStr = path[0] + "\n"
            if pathStr not in recentFileList:
                recentFileList.append(pathStr)
            
            tsneClass = TsnePlot(path[0])
            sheets = tsneClass.getSheets()
            tsneClass.setSheet(sheets[0])
            tsneClass.setOutputColumns("DM", "DM Duration")
            figure = tsneClass.getPlot()
            
            self.w = ClusteringAlgorithmWindow(figure)
            self.w.show()
            self.close()
            
    def open_instructions_window(self, checked):
        self.w = InstructionsWindow()
        self.w.show()
        self.close()
        
    def closeEvent(self, event):
        writeRecentFiles()
        self.close()
        
class InstructionsWindow(QWidget):
  
    def __init__(self):
        super().__init__()
        self.init_ui()
        
    def init_ui(self):
        self.resize(1500, 750)
        self.center()
        self.setStyleSheet(styles)
        self.setWindowTitle('')
        
        label1 = QLabel('<h1>Clustering Algorithm Instructions</h1>')
        label1.setAlignment(Qt.AlignHCenter)
        
        label2 = QLabel('<h3>1. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.</h3>')
        label2.setAlignment(Qt.AlignHCenter) 
        
        label3 = QLabel('<h3>2. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.</h3>')
        label3.setAlignment(Qt.AlignHCenter) 
        
        label4 = QLabel('<h3>3. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.</h3>')
        label4.setAlignment(Qt.AlignHCenter) 

        label5 = QLabel('<h3>4. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.</h3>')
        label5.setAlignment(Qt.AlignHCenter)
        
        layout = QVBoxLayout()
        layout.addWidget(label1)
        layout.addWidget(label2)
        layout.addWidget(label3)
        layout.addWidget(label4)
        layout.addWidget(label5)

        self.setLayout(layout)
        
        self.show()
        
    def center(self):
        # geometry of the main window
        qr = self.frameGeometry()

        # center point of screen
        cp = QDesktopWidget().availableGeometry().center()

        # move rectangle's center point to screen's center point
        qr.moveCenter(cp)

        # top left of rectangle becomes top left of window centering it
        self.move(qr.topLeft())
        
    def closeEvent(self, event):
        self.w = UI()
        self.w.show()
        self.close()
        
class ClusteringAlgorithmWindow(QWidget):
  
    def __init__(self, figure):
        super().__init__()
        self.init_ui(figure)
        
    def init_ui(self, figure):
        self.resize(1500, 750)
        self.center()
        self.setWindowTitle('')
        
        label1 = QLabel('<h1>The outputs of the algorithm are shown below</h1>')
        label1.setAlignment(Qt.AlignHCenter)
        
        html = '<html><body>'
        html += plotly.offline.plot(figure, output_type='div', include_plotlyjs='cdn')
        html += '</body></html>'

        # we create an instance of QWebEngineView and set the html code
        plot_widget = QWebEngineView()
        plot_widget.setHtml(html)

        layout = QVBoxLayout()
        layout.addWidget(label1)
        layout.addWidget(plot_widget)

        self.setLayout(layout)
        self.show()
        
    def center(self):
        # geometry of the main window
        qr = self.frameGeometry()

        # center point of screen
        cp = QDesktopWidget().availableGeometry().center()

        # move rectangle's center point to screen's center point
        qr.moveCenter(cp)

        # top left of rectangle becomes top left of window centering it
        self.move(qr.topLeft())
        
    def closeEvent(self, event):
        self.w = UI()
        self.w.show()
        self.close()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    ex = UI()
    sys.exit(app.exec_())